In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
import env


#RUN this if you are not using the docker image (use it cause takes too much memory otherwise)
from huggingface_hub import login
login(env.HF_LOGIN_TOKEN)

# ----------------------------
# Step 1: Connect to Qdrant
# ----------------------------
# Option 1: In-memory (no Docker)
client = QdrantClient(":memory:")

# # Option 2: Local Qdrant server
# client = QdrantClient("http://localhost:6333")




In [ ]:
# ----------------------------
# Step 2: Initialize model
# ----------------------------
model = SentenceTransformer("intfloat/multilingual-e5-large")
model.save("models/multilingual-e5-large") #save model locally (do only once)

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
import torch
#If you already saved the model locally and are using docker
client = QdrantClient(host="localhost", port=6333, prefer_grpc=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("models/multilingual-e5-large", device=device)

# Run docker container before
# docker run -d --name qdrant -p 6333:6333 -p 6334:6334 -v /home/Loris/EPFL/MA3/ML/project2/project2Rag/qdrant_storage:/qdrant/storage qdrant/qdrant:latest



In [1]:

# ----------------------------
# Step 3: Load articles from JSON (Polars)
# ----------------------------
from pathlib import Path
import json
import polars as pl

path = Path("../wikiextractor/articles_fr_withoutLinks.json")

ARTICLES_NUMBER = None  # None = load all; set an int to cap when sampling

def load_jsonl_to_df(path: Path, max_rows: int | None = None) -> pl.DataFrame:
    # If loading the full dataset, use Polars' fast NDJSON reader
    if max_rows is None:
        try:
            df = pl.read_ndjson(str(path))
        except Exception:
            # Fallback if the NDJSON reader hits malformed lines: manual parse
            rows = []
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    try:
                        rows.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
            df = pl.DataFrame(rows)
    else:
        # Limited load: manual parse up to max_rows
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i >= max_rows:
                    break
                try:
                    rows.append(json.loads(line))
                except json.JSONDecodeError:
                    continue
        df = pl.DataFrame(rows)

    # Ensure columns exist and normalize types/nulls
    cols = set(df.columns)

    if "id" in cols:
        # Cast id to Int64
        df = df.with_columns(pl.col("id").cast(pl.Int64))
    else:
        df = df.with_columns(pl.lit(None).cast(pl.Int64).alias("id"))

    if "title" in cols:
        df = df.with_columns(pl.col("title").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("title"))

    if "url" in cols:
        df = df.with_columns(pl.col("url").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("url"))

    if "text" in cols:
        df = df.with_columns(pl.col("text").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("text"))

    return df

df = load_jsonl_to_df(path, max_rows=ARTICLES_NUMBER)
print(f"✅ Loaded {len(df)} articles into a single Polars DataFrame")


✅ Loaded 4498441 articles into a single Polars DataFrame


In [2]:
#Write polars df to disk if needed
df.write_parquet("articles_fr_withoutLinks.parquet")

In [3]:
import polars as pl
import re
import html
from urllib.parse import unquote
#Extract href links from the articles with links and saves them to a df

BASE_URL = "https://fr.wikipedia.org/wiki/"
ANCHOR_RE = re.compile(r'<a\s+[^>]*href="([^"]+)"[^>]*>(.*?)</a>', re.IGNORECASE | re.DOTALL)

def extract_links_with_pos(text: str):
    if not text:
        return []
    unescaped = html.unescape(text)
    out = []
    search_from = 0
    for m in ANCHOR_RE.finditer(unescaped):
        href_raw = m.group(1)
        anchor = html.unescape(m.group(2))
        needle = f'&lt;a href="{href_raw}"'
        pos = text.find(needle, search_from)
        if pos == -1:
            pos = text.find("&lt;a ", search_from)
        if pos == -1:
            pos = 0
        search_from = pos + 1
        out.append({
            "full_url": BASE_URL + href_raw,
            "start_idx": pos,
            "anchor": anchor,
            "href_raw": href_raw,
            "href_decoded": unquote(href_raw),
        })
    return out

LINK_ITEM = pl.Struct([
    pl.Field("full_url", pl.Utf8),
    pl.Field("start_idx", pl.Int64),
    pl.Field("anchor", pl.Utf8),
    pl.Field("href_raw", pl.Utf8),
    pl.Field("href_decoded", pl.Utf8),
])
LINK_LIST = pl.List(LINK_ITEM)

def build_per_article_links(df: pl.DataFrame, max_rows: int | None = None) -> pl.DataFrame:
    if max_rows is not None:
        df = df.slice(0, min(max_rows, df.height))
    idx = (
        df.select(["id", "title", "text"])
          .with_columns(pl.col("text").cast(pl.Utf8).fill_null(""))
          .with_columns(links=pl.col("text").map_elements(extract_links_with_pos, return_dtype=LINK_LIST))
          .drop("text")
          .with_columns(
              links=pl.when(pl.col("links").is_null()).then(pl.lit([]).cast(LINK_LIST)).otherwise(pl.col("links")),
              link_count=pl.col("links").list.len()
          )
    )
    return idx

# Example usage on your Polars df:
per_article = build_per_article_links(df, max_rows=10000)
# print(per_article.select(["id", "title", "link_count"]).head(10))

# Inspect one article: all links and their positions in a single row
article_id = 3
row = per_article.filter(pl.col("id") == article_id)
links = row.select("links").to_series().to_list()[0] if row.height else []
print(links)
# for l in links[:20]:
#     print(f"{l['full_url']} - {l['start_idx']}")

[{'full_url': 'https://fr.wikipedia.org/wiki/Moulins%20%28Allier%29', 'start_idx': 25, 'anchor': 'Moulins', 'href_raw': 'Moulins%20%28Allier%29', 'href_decoded': 'Moulins (Allier)'}, {'full_url': 'https://fr.wikipedia.org/wiki/Allier%20%28d%C3%A9partement%29', 'start_idx': 83, 'anchor': 'Allier', 'href_raw': 'Allier%20%28d%C3%A9partement%29', 'href_decoded': 'Allier (département)'}, {'full_url': 'https://fr.wikipedia.org/wiki/Ch%C3%A2teaumeillant', 'start_idx': 162, 'anchor': 'Châteaumeillant', 'href_raw': 'Ch%C3%A2teaumeillant', 'href_decoded': 'Châteaumeillant'}, {'full_url': 'https://fr.wikipedia.org/wiki/Cher%20%28d%C3%A9partement%29', 'start_idx': 226, 'anchor': 'Cher', 'href_raw': 'Cher%20%28d%C3%A9partement%29', 'href_decoded': 'Cher (département)'}, {'full_url': 'https://fr.wikipedia.org/wiki/Philologie', 'start_idx': 296, 'anchor': 'philologue', 'href_raw': 'Philologie', 'href_decoded': 'Philologie'}, {'full_url': 'https://fr.wikipedia.org/wiki/liste%20de%20linguistes', 'start

In [ ]:
# Assumes df, LINK_ITEM, LINK_LIST, and build_per_article_links are already defined in previous cells

per_article = build_per_article_links(df, max_rows=None)
df_out = df.join(per_article.select(["id", "links", "link_count"]), on="id", how="left")
df_out.write_parquet("articles_fr_withLinks_withHyperlinks.parquet")

# Optional preview
print(df_out.select(["id", "title", "link_count"]).head(10))

In [4]:
import polars as pl
import pyarrow.parquet as pq
import pyarrow as pa

# Assumes extract_links_with_pos, LINK_ITEM, LINK_LIST, and build_per_article_links(df, max_rows)
# are already defined in previous cells

def export_with_links_streaming(parquet_in: str, parquet_out: str, max_rows: int | None = None):
    pf = pq.ParquetFile(parquet_in)
    writer = None
    processed = 0

    for rg in range(pf.num_row_groups):
        if max_rows is not None and processed >= max_rows:
            break

        table = pf.read_row_group(rg, columns=["id", "title", "text"])
        sub_df = pl.from_arrow(table).with_columns([
            pl.col("id").cast(pl.Int64),
            pl.col("title").cast(pl.Utf8).fill_null(""),
            pl.col("text").cast(pl.Utf8).fill_null(""),
        ])

        remaining = None if max_rows is None else max_rows - processed
        if remaining is not None and sub_df.height > remaining:
            sub_df = sub_df.slice(0, remaining)

        per_article = build_per_article_links(sub_df, max_rows=None)
        sub_out = sub_df.join(per_article.select(["id", "links", "link_count"]), on="id", how="left")

        arrow_batch = sub_out.to_arrow()
        if writer is None:
            writer = pq.ParquetWriter(parquet_out, arrow_batch.schema, compression="zstd")
        writer.write_table(arrow_batch)

        processed += sub_df.height

    if writer is not None:
        writer.close()

# Example usage (streams input and produces output shards in-place):
export_with_links_streaming(
    parquet_in="articles_fr_withLinks.parquet",
    parquet_out="articles_fr_withLinks_withHyperlinks.parquet",
    max_rows=None  # or set an int cap to limit total processed rows
)

In [1]:
import polars as pl
df = pl.read_parquet("articles_fr_withoutLinks.parquet")
df2 = pl.read_parquet("articles_fr_withLinks_withHyperlinks.parquet")


In [3]:
print(df.head())
print(df2.head())
print(df.schema)
print(df2.schema)
print(df.shape)
print(df2.shape)

# Schema([('id', Int64), ('revid', String), ('url', String), ('title', String), ('text', String)])
# Schema([('id', Int64), ('title', String), ('text', String), ('links', List(Struct({'full_url': String, 'start_idx': Int64, 'anchor': String, 'href_raw': String, 'href_decoded': String}))), ('link_count', UInt32)])

# print(df.head()["links"])
# print(df.head()["text"])


# ┌─────┬─────────┬────────────────────────────┬────────────────────────┬────────────────────────────┐
# │ id  ┆ revid   ┆ url                        ┆ title                  ┆ text                       │
# │ --- ┆ ---     ┆ ---                        ┆ ---                    ┆ ---                        │
# │ i64 ┆ str     ┆ str                        ┆ str                    ┆ str                        │
# ╞═════╪═════════╪════════════════════════════╪════════════════════════╪════════════════════════════╡
# │ 3   ┆ 316223  ┆ https://fr.wikipedia.org/w ┆ Antoine Meillet        ┆ Antoine Meillet, né le à   │
# │     ┆         ┆ iki?…                      ┆                        ┆ Mouli…                     │
# │ 7   ┆ 5256901 ┆ https://fr.wikipedia.org/w ┆ Algèbre linéaire       ┆ L’algèbre linéaire est la  │
# │     ┆         ┆ iki?…                      ┆                        ┆ bran…                      │
# │ 9   ┆ 1430140 ┆ https://fr.wikipedia.org/w ┆ Algèbre générale       ┆ L'algèbre générale, ou     │
# │     ┆         ┆ iki?…                      ┆                        ┆ algèbre…                   │
# │ 10  ┆ 1012    ┆ https://fr.wikipedia.org/w ┆ Algorithmique          ┆ L'algorithmique est        │
# │     ┆         ┆ iki?…                      ┆                        ┆ l'étude et…                │
# │ 11  ┆ 202320  ┆ https://fr.wikipedia.org/w ┆ Politique en Argentine ┆ L'Argentine est une        │
# │     ┆         ┆ iki?…                      ┆                        ┆ république…                │
# └─────┴─────────┴────────────────────────────┴────────────────────────┴────────────────────────────┘
# shape: (5, 5)
# ┌─────┬────────────────────────┬───────────────────────────┬──────────────────────────┬────────────┐
# │ id  ┆ title                  ┆ text                      ┆ links                    ┆ link_count │
# │ --- ┆ ---                    ┆ ---                       ┆ ---                      ┆ ---        │
# │ i64 ┆ str                    ┆ str                       ┆ list[struct[5]]          ┆ u32        │
# ╞═════╪════════════════════════╪═══════════════════════════╪══════════════════════════╪════════════╡
# │ 3   ┆ Antoine Meillet        ┆ Antoine Meillet, né le à  ┆ [{"https://fr.wikipedia. ┆ 65         │
# │     ┆                        ┆ &lt;a…                    ┆ org/wi…                  ┆            │
# ...
# │     ┆                        ┆ l'étude et…               ┆ org/wi…                  ┆            │
# │ 11  ┆ Politique en Argentine ┆ L'&lt;a                   ┆ [{"https://fr.wikipedia. ┆ 78         │
# │     ┆                        ┆ href="Argentine"&gt;Ar…   ┆ org/wi…                  ┆            │
# └─────┴────────────────────────┴───────────────────────────┴──────────────────────────┴────────────┘



shape: (5, 5)
┌─────┬─────────┬────────────────────────────┬────────────────────────┬────────────────────────────┐
│ id  ┆ revid   ┆ url                        ┆ title                  ┆ text                       │
│ --- ┆ ---     ┆ ---                        ┆ ---                    ┆ ---                        │
│ i64 ┆ str     ┆ str                        ┆ str                    ┆ str                        │
╞═════╪═════════╪════════════════════════════╪════════════════════════╪════════════════════════════╡
│ 3   ┆ 316223  ┆ https://fr.wikipedia.org/w ┆ Antoine Meillet        ┆ Antoine Meillet, né le à   │
│     ┆         ┆ iki?…                      ┆                        ┆ Mouli…                     │
│ 7   ┆ 5256901 ┆ https://fr.wikipedia.org/w ┆ Algèbre linéaire       ┆ L’algèbre linéaire est la  │
│     ┆         ┆ iki?…                      ┆                        ┆ bran…                      │
│ 9   ┆ 1430140 ┆ https://fr.wikipedia.org/w ┆ Algèbre générale       ┆ L'alg

In [2]:
import polars as pl

with_links = (
    pl.scan_parquet("articles_fr_withLinks_withHyperlinks.parquet")
      .select([pl.col("id").cast(pl.Int64), pl.col("title"), pl.col("text"), pl.col("links"), pl.col("link_count")])
)
without_links = (
    pl.scan_parquet("articles_fr_withoutLinks.parquet")
      .select([pl.col("id").cast(pl.Int64), pl.col("text").alias("text_plain")])
)

inner = (
    with_links.join(without_links, on="id", how="inner")
    .select(pl.len().alias("n"))
    .collect(engine="streaming")["n"][0]
)
left_unmatched = (
    with_links.join(without_links, on="id", how="anti")
    .select(pl.len().alias("n"))
    .collect(engine="streaming")["n"][0]
)
right_unmatched = (
    without_links.join(with_links, on="id", how="anti")
    .select(pl.len().alias("n"))
    .collect(engine="streaming")["n"][0]
)

print("inner matches:", inner, "left-only:", left_unmatched, "right-only:", right_unmatched)

inner matches: 4498441 left-only: 0 right-only: 0


In [1]:
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq

with_links_path = "articles_fr_withLinks_withHyperlinks.parquet"
without_links_path = "articles_fr_withoutLinks.parquet"
out_path = "articles_fr_merged.parquet"

pf_with = pq.ParquetFile(with_links_path)
pf_plain = pq.ParquetFile(without_links_path)

n_with = pf_with.metadata.num_rows
n_plain = pf_plain.metadata.num_rows
if n_with != n_plain:
    raise ValueError(f"Row count mismatch: {n_with} vs {n_plain}")

batch_size = 200_000
writer = None

for b_with, b_plain in zip(
    pf_with.iter_batches(batch_size=batch_size, columns=["id", "title", "text", "links", "link_count"]),
    pf_plain.iter_batches(batch_size=batch_size, columns=["text"]),
):
    df_with = pl.from_arrow(b_with)
    df_plain = pl.from_arrow(b_plain).rename({"text": "text_withoutHref"})
    out = pl.concat([df_with, df_plain], how="horizontal")
    arrow_tbl = out.to_arrow()
    if writer is None:
        writer = pq.ParquetWriter(out_path, arrow_tbl.schema, compression="zstd")
    writer.write_table(arrow_tbl)

if writer is not None:
    writer.close()

print(pl.scan_parquet(out_path).select(["id", "title", "link_count"]).limit(10).collect(engine="streaming"))

shape: (10, 3)
┌─────┬───────────────────────────────┬────────────┐
│ id  ┆ title                         ┆ link_count │
│ --- ┆ ---                           ┆ ---        │
│ i64 ┆ str                           ┆ u32        │
╞═════╪═══════════════════════════════╪════════════╡
│ 3   ┆ Antoine Meillet               ┆ 65         │
│ 7   ┆ Algèbre linéaire              ┆ 111        │
│ 9   ┆ Algèbre générale              ┆ 15         │
│ 10  ┆ Algorithmique                 ┆ 101        │
│ 11  ┆ Politique en Argentine        ┆ 78         │
│ 12  ┆ Armée républicaine irlandaise ┆ 2          │
│ 15  ┆ Autriche                      ┆ 267        │
│ 16  ┆ Arc de triomphe de l'Étoile   ┆ 191        │
│ 18  ┆ Arsène Lupin                  ┆ 317        │
│ 19  ┆ Algorithme                    ┆ 71         │
└─────┴───────────────────────────────┴────────────┘


In [ ]:

import polars as pl
df = pl.read_parquet("articles_fr_merged.parquet")
print(df.head())
print(df.shape)
print(df["title"][2])
print(df["title"][32])

shape: (5, 6)
┌─────┬───────────────────┬───────────────────┬───────────────────┬────────────┬───────────────────┐
│ id  ┆ title             ┆ text              ┆ links             ┆ link_count ┆ text_withoutHref  │
│ --- ┆ ---               ┆ ---               ┆ ---               ┆ ---        ┆ ---               │
│ i64 ┆ str               ┆ str               ┆ list[struct[5]]   ┆ u32        ┆ str               │
╞═════╪═══════════════════╪═══════════════════╪═══════════════════╪════════════╪═══════════════════╡
│ 3   ┆ Antoine Meillet   ┆ Antoine Meillet,  ┆ [{"https://fr.wik ┆ 65         ┆ Antoine Meillet,  │
│     ┆                   ┆ né le à &lt;a…    ┆ ipedia.org/wi…    ┆            ┆ né le à Mouli…    │
│ 7   ┆ Algèbre linéaire  ┆ L’algèbre         ┆ [{"https://fr.wik ┆ 111        ┆ L’algèbre         │
│     ┆                   ┆ linéaire est la   ┆ ipedia.org/wi…    ┆            ┆ linéaire est la   │
│     ┆                   ┆ bran…             ┆                   ┆          

In [1]:
import polars as pl

with_links = (
    pl.scan_parquet("articles_fr_withLinks_withHyperlinks.parquet")
      .select([
          pl.col("id").cast(pl.Int64),
          pl.col("title"),
          pl.col("text"),
          pl.col("links"),
          pl.col("link_count"),
      ])
)

without_links = (
    pl.scan_parquet("articles_fr_withoutLinks.parquet")
      .select([
          pl.col("id").cast(pl.Int64),
          pl.col("text").alias("text_plain"),
      ])
)

merged = with_links.join(without_links, on="id", how="left")
try:
    merged.sink_parquet("articles_fr_merged.parquet")
except AttributeError:
    merged.collect(streaming=True).write_parquet("articles_fr_merged.parquet")

In [ ]:
ar

In [ ]:
# Load vector size from model
vector_size = model.get_sentence_embedding_dimension()



In [ ]:
#Warning: only create the collection once ! Otherwise you will loose the content of it

# collection_name = "wikipedia_fr"
# if not client.collection_exists(collection_name):
#     client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=vector_size, distance="Cosine"),
#     )
# else:
#     print(f"Collection '{collection_name}' already exists; skipping creation.")

In [1]:
#GO from here to load new articles into Qdrant Onwards

import pyarrow.parquet as pq

#load file from parquet (if you saved it once previously)
pf = pq.ParquetFile("articles_fr_withLinks.parquet")
rowgroup_batch = 512
encode_batch = 64


In [6]:
from qdrant_client.http import models as rest
#get articles count from Qdrant DB 

collection_name = "wikipedia_fr"
total = client.count(collection_name=collection_name, exact=True).count
print(f"Total articles stored: {total}")

Total articles stored: 30208


In [ ]:
#Recover a snapshot from the Qdrant db (should not be needed)
client.recover_snapshot(collection_name="wikipedia_fr", location="http://localhost:6333/collections/wikipedia_fr/snapshots/<snapshot_name>")

In [5]:
import polars as pl

# Process Articles into the Qdrant DB (warning: takes shit ton of time and ressources)

ARTICLES_NUMBER=4000

new_processed = 0
max_rows = ARTICLES_NUMBER

for rg in range(pf.num_row_groups):
    if max_rows is not None and new_processed >= max_rows:
        break
    table = pf.read_row_group(rg, columns=["id", "title", "url", "text"])
    df = pl.from_arrow(table).with_columns([
        pl.col("id").cast(pl.Int64),
        pl.col("title").fill_null(""),
        pl.col("url").fill_null(""),
        pl.col("text").fill_null("")
    ])
    n = df.height
    remaining = None if max_rows is None else max_rows - new_processed
    for start in range(0, n, rowgroup_batch):
        length = min(rowgroup_batch, n - start)
        if remaining is not None:
            length = min(length, remaining)
        sub = df.slice(start, length)
        ids = sub["id"].to_list()
        texts = sub["text"].to_list()
        titles = sub["title"].to_list()
        urls = sub["url"].to_list()

        existing = client.retrieve(
            collection_name="wikipedia_fr",
            ids=[int(i) for i in ids],
            with_payload=False,
            with_vectors=False
        )
        existing_ids = {p.id for p in existing}
        missing_idx = [i for i, pid in enumerate(ids) if int(pid) not in existing_ids]
        if not missing_idx:
            continue

        ids_m = [ids[i] for i in missing_idx]
        texts_m = [texts[i] for i in missing_idx]
        titles_m = [titles[i] for i in missing_idx]
        urls_m = [urls[i] for i in missing_idx]

        vectors = model.encode(texts_m, batch_size=encode_batch, convert_to_numpy=True, normalize_embeddings=True)
        points = [
            PointStruct(
                id=int(ids_m[i]),
                vector=vectors[i].tolist(),
                payload={"id": int(ids_m[i]), "title": titles_m[i], "url": urls_m[i], "text": texts_m[i]},
            )
            for i in range(len(ids_m))
        ]
        client.upsert(collection_name="wikipedia_fr", points=points, wait=True)
        new_processed += len(ids_m)
        if max_rows is not None and new_processed >= max_rows:
            break

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
import polars as pl
from typing import Optional

#Query Qdrant DB to rebuild the polars df (warning: untested)

def rebuild_df_from_qdrant(
    client: QdrantClient,
    collection_name: str,
    max_rows: Optional[int] = 1000,
    batch_limit: int = 1000,
) -> pl.DataFrame:
    rows = []
    next_page = None

    selector = rest.PayloadSelectorInclude(include=["id", "title", "url", "text"])

    while True:
        points, next_page = client.scroll(
            collection_name=collection_name,
            limit=batch_limit,
            with_payload=True,
            with_vectors=False,
            payload_selector=selector,
            offset=next_page,
        )
        if not points:
            break

        for p in points:
            pid = p.payload.get("id", p.id)
            rows.append({
                "id": pid,
                "title": p.payload.get("title", ""),
                "url": p.payload.get("url", ""),
                "text": p.payload.get("text", ""),
            })
            if max_rows is not None and len(rows) >= max_rows:
                break

        if max_rows is not None and len(rows) >= max_rows:
            break
        if next_page is None:
            break

    df = pl.DataFrame(rows).with_columns([
        pl.col("id").cast(pl.Int64),
        pl.col("title").cast(pl.Utf8).fill_null(""),
        pl.col("url").cast(pl.Utf8).fill_null(""),
        pl.col("text").cast(pl.Utf8).fill_null(""),
    ])
    return df


In [ ]:
import polars as pl

#Query DB with one keyword for cosine similarity (warning : NANs issues gives 1.0 score)

query_title = "Math"

qdf = df.filter(pl.col("title").str.to_lowercase() == query_title.lower())
query_text = qdf.select("text").to_series()[0] if qdf.height > 0 else query_title

# E5 requires the 'query:' prefix for queries; documents were encoded as 'passage:' during ingestion
query_vector = model.encode([f"query: {query_text}"], normalize_embeddings=True)[0]

results = client.search(
    collection_name="wikipedia_fr",
    query_vector=query_vector,
    limit=5
)

print(f"\n🔍 Top matches for '{query_title}':")
for r in results:
    print(f"• {r.payload.get('title')} (score={r.score:.3f})")


🔍 Top matches for 'Math':
• Nombre entier (score=0.824)
• Série (mathématiques) (score=0.818)
• Distance (mathématiques) (score=0.813)
• Nombre (score=0.812)
• Géométrie arithmétique (score=0.811)


/tmp/ipykernel_1272/1232709101.py:11: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
import polars as pl
import numpy as np

# Function to encode whole article text into vector 
# Then query the db with the article embedding to get the k most similar articles

def encode_article_text(text):
    words = text.split()
    if not words:
        return np.zeros(model.get_sentence_embedding_dimension(), dtype=np.float32)
    chunks = [" ".join(words[i:i+256]) for i in range(0, len(words), 256)]
    vecs = model.encode([f"passage: {c}" for c in chunks], normalize_embeddings=True)
    return np.mean(vecs, axis=0)

mode = "ids"  # "all" or "ids"
ids_to_process = [189, 205]
k = 5

ids_all = df.select(pl.col("id").cast(pl.Int64)).to_series().to_list()
ids = ids_all if mode == "all" else [i for i in ids_to_process if i in set(ids_all)]

for article_id in ids:
    title_row = df.filter(pl.col("id") == int(article_id)).select("title")
    title = title_row.to_series()[0] if title_row.height > 0 else "(unknown)"
    try:
        points = client.retrieve(collection_name="wikipedia_fr", ids=[int(article_id)], with_payload=True, with_vectors=False)
        if points:
            results = client.recommend(collection_name="wikipedia_fr", positive=[int(article_id)], limit=k)
        else:
            raise ValueError("missing")
    except Exception:
        row = df.filter(pl.col("id") == int(article_id))
        if row.height == 0:
            continue
        text = row.select(pl.col("text").fill_null("")).to_series()[0]
        vec = encode_article_text(text)
        results = client.search(collection_name="wikipedia_fr", query_vector=vec, limit=k)
    print(f"\n🔎 Similar articles for: {title} (id={int(article_id)})")
    for r in results:
        print(f"  • {r.payload.get('title')} (id={r.payload.get('id')}, score={r.score:.3f})")


🔎 Similar articles for: Atome (id=189)
  • Proton (id=2414, score=0.939)
  • Liaison chimique (id=22131, score=0.939)
  • Élément chimique (id=15349, score=0.936)
  • Électron (id=6716, score=0.935)
  • Particule élémentaire (id=23547, score=0.934)


/tmp/ipykernel_33189/278556868.py:25: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.recommend(collection_name="wikipedia_fr", positive=[int(article_id)], limit=k)


: 

In [ ]:
import polars as pl

#Export articles to csv for further use

cols = ["id", "title", "url", "text"]
df.select(cols).write_csv("articles.csv")

In [ ]:
import polars as pl
import pyarrow.parquet as pq
import pyarrow.csv as pacsv

pf = pq.ParquetFile("articles_fr_withLinks.parquet")
cols = ["id", "title", "url", "text"]
ARTICLES_NUMBER = 100000
processed = 0
first = True
output_path = "articles_stream.csv"

for rg in range(pf.num_row_groups):
    if ARTICLES_NUMBER is not None and processed >= ARTICLES_NUMBER:
        break
    table = pf.read_row_group(rg, columns=cols)
    df_rg = pl.from_arrow(table).with_columns([
        pl.col("id").cast(pl.Int64),
        pl.col("title").fill_null(""),
        pl.col("url").fill_null(""),
        pl.col("text").fill_null("")
    ])
    remaining = None if ARTICLES_NUMBER is None else ARTICLES_NUMBER - processed
    if remaining is not None and df_rg.height > remaining:
        df_rg = df_rg.slice(0, remaining)
    mode = "wb" if first else "ab"
    with open(output_path, mode) as f:
        pacsv.write_csv(df_rg.to_arrow(), f, write_options=pacsv.WriteOptions(include_header=first))
    processed += df_rg.height
    first = False

In [6]:
import polars as pl
import pyarrow.parquet as pq
from qdrant_client.models import PointStruct

def export_points_df(df: pl.DataFrame, client, collection_name: str, model, encode_batch: int = 64, upsert_batch: int = 512, max_rows: int | None = None):
    n = df.height if max_rows is None else min(df.height, max_rows)
    processed = 0
    while processed < n:
        length = min(upsert_batch, n - processed)
        sub = df.slice(processed, length)
        ids = sub["id"].cast(pl.Int64).to_list()
        titles = sub["title"].fill_null("").to_list()
        urls = sub["url"].fill_null("").to_list()
        texts = sub["text"].fill_null("").to_list()
        vectors = model.encode(texts, batch_size=encode_batch, convert_to_numpy=True, normalize_embeddings=True)
        points = [
            PointStruct(
                id=int(ids[i]),
                vector=vectors[i].tolist(),
                payload={"id": int(ids[i]), "title": titles[i], "url": urls[i], "text": texts[i]},
            )
            for i in range(len(ids))
        ]
        client.upsert(collection_name=collection_name, points=points, wait=True)
        processed += length

In [7]:
import polars as pl
import pyarrow.parquet as pq
from qdrant_client.models import PointStruct

def export_points_parquet(parquet_path: str, client, collection_name: str, model, encode_batch: int = 64, upsert_batch: int = 512, max_rows: int | None = None):
    pf = pq.ParquetFile(parquet_path)
    processed = 0
    for rg in range(pf.num_row_groups):
        if max_rows is not None and processed >= max_rows:
            break
        table = pf.read_row_group(rg, columns=["id", "title", "url", "text"])
        df = pl.from_arrow(table).with_columns([
            pl.col("id").cast(pl.Int64),
            pl.col("title").fill_null(""),
            pl.col("url").fill_null(""),
            pl.col("text").fill_null("")
        ])
        remaining = None if max_rows is None else max_rows - processed
        n = df.height if remaining is None else min(df.height, remaining)
        inner = 0
        while inner < n:
            length = min(upsert_batch, n - inner)
            sub = df.slice(inner, length)
            ids = sub["id"].to_list()
            titles = sub["title"].to_list()
            urls = sub["url"].to_list()
            texts = sub["text"].to_list()
            vectors = model.encode(texts, batch_size=encode_batch, convert_to_numpy=True, normalize_embeddings=True)
            points = [
                PointStruct(
                    id=int(ids[i]),
                    vector=vectors[i].tolist(),
                    payload={"id": int(ids[i]), "title": titles[i], "url": urls[i], "text": texts[i]},
                )
                for i in range(len(ids))
            ]
            client.upsert(collection_name=collection_name, points=points, wait=True)
            processed += length
            inner += length

In [3]:
import polars as pl
import numpy as np
from qdrant_client.http import models as rest
from typing import Optional
import csv
from tqdm import tqdm

def export_similarity_graph(
    client,
    collection_name: str = "wikipedia_fr",
    output_path: str = "similarity_links.csv",
    k: int = 10,
    max_articles: Optional[int] = None,
    batch_size: int = 1000,
    resume_from_id: Optional[int] = None
):
    """
    Export k-NN similarity graph from Qdrant database efficiently.
    
    Args:
        client: Qdrant client
        collection_name: Name of the collection
        output_path: CSV output file path
        k: Number of similar articles to find for each article
        max_articles: Maximum number of articles to process (None for all)
        batch_size: Batch size for scrolling through collection
        resume_from_id: Resume from specific article ID (for interrupted runs)
    """
    print(f"🚀 Starting similarity graph export...")
    print(f"   Collection: {collection_name}")
    print(f"   Output: {output_path}")
    print(f"   k-NN: {k}")
    print(f"   Max articles: {max_articles or 'All'}")
    
    # Open CSV file for writing
    mode = 'a' if resume_from_id else 'w'
    with open(output_path, mode, newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write header only if starting fresh
        if not resume_from_id:
            writer.writerow(['source_id', 'target_id', 'score'])
        
        # Iterator to get all article IDs from Qdrant
        processed_count = 0
        next_page = None
        
        # Skip to resume point if specified
        if resume_from_id:
            print(f"   Resuming from article ID: {resume_from_id}")
        
        while True:
            # Scroll through collection to get article IDs
            points, next_page = client.scroll(
                collection_name=collection_name,
                limit=batch_size,
                with_payload=True,
                with_vectors=False,
                offset=next_page,
            )
            
            if not points:
                break
            
            # Process each article in the batch
            for point in tqdm(points, desc=f"Processing batch (total: {processed_count})"):
                source_id = point.payload.get("id", point.id)
                
                # Skip if resuming and haven't reached resume point
                if resume_from_id and int(source_id) < resume_from_id:
                    continue
                
                try:
                    # Use Qdrant's recommend API for efficient similarity search
                    # This leverages the stored vectors without re-encoding
                    similar_results = client.recommend(
                        collection_name=collection_name,
                        positive=[int(source_id)],
                        limit=k + 1,  # +1 because it includes the source article
                        with_payload=True
                    )
                    
                    # Write similarity edges to CSV
                    for result in similar_results:
                        target_id = result.payload.get("id", result.id)
                        
                        # Skip self-similarity
                        if int(target_id) == int(source_id):
                            continue
                        
                        # Write edge: source_id -> target_id with similarity score
                        writer.writerow([source_id, target_id, f"{result.score:.6f}"])
                    
                except Exception as e:
                    print(f"   ⚠️  Error processing article {source_id}: {e}")
                    continue
                
                processed_count += 1
                
                # Check if we've reached the maximum
                if max_articles and processed_count >= max_articles:
                    print(f"\n✅ Reached maximum articles limit: {max_articles}")
                    return processed_count
                
                # Flush every 100 articles for safety
                if processed_count % 100 == 0:
                    csvfile.flush()
            
            # Break if no more pages
            if next_page is None:
                break
    
    print(f"\n✅ Similarity graph export completed!")
    print(f"   Processed articles: {processed_count:,}")
    print(f"   Output file: {output_path}")
    
    return processed_count

In [4]:
def export_similarity_graph_hybrid(
    client,
    df: pl.DataFrame,
    collection_name: str = "wikipedia_fr",
    output_path: str = "similarity_links_hybrid.csv",
    k: int = 10,
    max_articles: Optional[int] = None
):
    """
    Export similarity graph using hybrid approach:
    - Use Qdrant recommend API for articles in the database
    - Use encode_article_text + search for articles not in database
    """
    print(f"🔄 Starting hybrid similarity export...")
    
    # Get list of article IDs to process
    article_ids = df.select("id").to_series().to_list()
    if max_articles:
        article_ids = article_ids[:max_articles]
    
    with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['source_id', 'target_id', 'score'])
        
        for i, article_id in enumerate(tqdm(article_ids, desc="Processing articles")):
            try:
                # Try Qdrant recommend API first (faster)
                points = client.retrieve(
                    collection_name=collection_name, 
                    ids=[int(article_id)], 
                    with_payload=True, 
                    with_vectors=False
                )
                
                if points:
                    # Article exists in Qdrant - use recommend API
                    results = client.recommend(
                        collection_name=collection_name,
                        positive=[int(article_id)],
                        limit=k + 1
                    )
                else:
                    # Article not in Qdrant - encode and search
                    row = df.filter(pl.col("id") == int(article_id))
                    if row.height == 0:
                        continue
                    
                    text = row.select(pl.col("text_withoutHref").fill_null("")).to_series()[0]
                    vec = encode_article_text(text)
                    results = client.search(
                        collection_name=collection_name, 
                        query_vector=vec, 
                        limit=k
                    )
                
                # Write results
                for result in results:
                    target_id = result.payload.get("id", result.id)
                    if int(target_id) != int(article_id):  # Skip self
                        writer.writerow([article_id, target_id, f"{result.score:.6f}"])
                        
            except Exception as e:
                print(f"Error processing article {article_id}: {e}")
                continue
            
            # Flush periodically
            if i % 100 == 0:
                csvfile.flush()
    
    print(f"✅ Hybrid export completed: {output_path}")

In [5]:
def extract_hyperlink_graph(df: pl.DataFrame, output_path: str = "hyperlink_graph.csv"):
    """
    Extract the actual hyperlink graph from Wikipedia articles.
    This creates the ground truth graph of existing links.
    """
    print("🔗 Extracting hyperlink graph from articles...")
    
    # Create a mapping from Wikipedia URLs to article IDs
    url_to_id = {}
    for row in df.select(["id", "title"]).iter_rows(named=True):
        # Create potential Wikipedia URL patterns for this article
        title = row["title"].replace(" ", "_")
        url_patterns = [
            title,
            title.replace("_", "%20"),
            title.replace("_", " ")
        ]
        for pattern in url_patterns:
            url_to_id[pattern] = row["id"]
    
    hyperlinks = []
    
    for row in tqdm(df.iter_rows(named=True), total=len(df), desc="Processing articles"):
        source_id = row["id"]
        links = row.get("links", []) or []
        
        for link in links:
            href_decoded = link.get("href_decoded", "")
            
            # Try to map the link to an article ID
            target_id = None
            
            # Direct lookup
            if href_decoded in url_to_id:
                target_id = url_to_id[href_decoded]
            else:
                # Try variations
                for pattern in [href_decoded.replace("_", " "), href_decoded.replace("%20", " ")]:
                    if pattern in url_to_id:
                        target_id = url_to_id[pattern]
                        break
            
            if target_id and target_id != source_id:
                hyperlinks.append({
                    "source_id": source_id,
                    "target_id": target_id,
                    "anchor_text": link.get("anchor", ""),
                    "href_decoded": href_decoded
                })
    
    # Convert to DataFrame and save
    hyperlink_df = pl.DataFrame(hyperlinks)
    hyperlink_df.write_csv(output_path)
    
    print(f"✅ Extracted {len(hyperlink_df):,} hyperlinks")
    print(f"   Unique source articles: {hyperlink_df.select('source_id').n_unique():,}")
    print(f"   Unique target articles: {hyperlink_df.select('target_id').n_unique():,}")
    print(f"   Output: {output_path}")
    
    return hyperlink_df

In [12]:
# Example 1: Export similarity graph from Qdrant (fastest)
processed = export_similarity_graph(
    client=client,
    collection_name="wikipedia_fr",
    output_path="similarity_links_qdrant.csv",
    k=10,  # Find 10 most similar articles for each
    max_articles=1000,  # Start with 1000 articles for testing, set to None for all
    batch_size=100,
    resume_from_id=None  # Set to an ID to resume interrupted runs
)

print(f"\n📊 Exported {processed:,} articles with their similarity links")

🚀 Starting similarity graph export...
   Collection: wikipedia_fr
   Output: similarity_links_qdrant.csv
   k-NN: 10
   Max articles: 1000


Processing batch (total: 0):   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_829/3566637825.py:76: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar_results = client.recommend(
Processing batch (total: 900):  99%|█████████▉| 99/100 [00:02<00:00, 45.30it/s]


✅ Reached maximum articles limit: 1000

📊 Exported 1,000 articles with their similarity links


In [14]:
# Example 2: Hybrid approach for all articles
# Load your merged DataFrame if not already loaded
df = pl.read_parquet("articles_fr_merged.parquet")
print(f"Loaded {len(df):,} articles")

# Run hybrid export for a subset first (testing)
export_similarity_graph_hybrid(
    client=client,
    df=df.head(500),  # Start with 500 articles for testing
    collection_name="wikipedia_fr",
    output_path="similarity_links_hybrid_test.csv",
    k=10,
    max_articles=500
)

# Preview results
sim_df = pl.read_csv("similarity_links_hybrid_test.csv")
print(sim_df.head(10))
print(f"\nTotal similarity edges: {len(sim_df):,}")
print(f"Unique source articles: {sim_df.select('source_id').n_unique()}")
print(f"Average edges per article: {len(sim_df) / sim_df.select('source_id').n_unique():.1f}")

Loaded 4,498,441 articles
🔄 Starting hybrid similarity export...


Processing articles:   0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_829/4227211400.py:37: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.recommend(
Processing articles: 100%|██████████| 500/500 [00:23<00:00, 21.53it/s]


✅ Hybrid export completed: similarity_links_hybrid_test.csv
shape: (10, 3)
┌───────────┬───────────┬──────────┐
│ source_id ┆ target_id ┆ score    │
│ ---       ┆ ---       ┆ ---      │
│ i64       ┆ i64       ┆ f64      │
╞═══════════╪═══════════╪══════════╡
│ 3         ┆ 37999     ┆ 0.904169 │
│ 3         ┆ 251       ┆ 0.901769 │
│ 3         ┆ 13418     ┆ 0.901407 │
│ 3         ┆ 13779     ┆ 0.901042 │
│ 3         ┆ 12434     ┆ 0.900641 │
│ 3         ┆ 13419     ┆ 0.900595 │
│ 3         ┆ 34222     ┆ 0.89858  │
│ 3         ┆ 1647      ┆ 0.896724 │
│ 3         ┆ 23162     ┆ 0.896583 │
│ 3         ┆ 41558     ┆ 0.895497 │
└───────────┴───────────┴──────────┘

Total similarity edges: 5,500
Unique source articles: 500
Average edges per article: 11.0


In [15]:
# Example 3: Extract hyperlink graph
hyperlink_graph = extract_hyperlink_graph(df.head(1000))  # Start with subset
print("\n📊 Hyperlink graph preview:")
print(hyperlink_graph.head(10))

🔗 Extracting hyperlink graph from articles...


Processing articles: 100%|██████████| 1000/1000 [00:00<00:00, 3455.67it/s]

✅ Extracted 5,383 hyperlinks
   Unique source articles: 653
   Unique target articles: 510
   Output: hyperlink_graph.csv

📊 Hyperlink graph preview:
shape: (10, 4)
┌───────────┬───────────┬───────────────────────┬───────────────────────┐
│ source_id ┆ target_id ┆ anchor_text           ┆ href_decoded          │
│ ---       ┆ ---       ┆ ---                   ┆ ---                   │
│ i64       ┆ i64       ┆ str                   ┆ str                   │
╞═══════════╪═══════════╪═══════════════════════╪═══════════════════════╡
│ 3         ┆ 33        ┆ Allier                ┆ Allier (département)  │
│ 3         ┆ 548       ┆ Cher                  ┆ Cher (département)    │
│ 3         ┆ 33        ┆ bourbonnaise          ┆ Allier (département)  │
│ 3         ┆ 548       ┆ Cher                  ┆ Cher (département)    │
│ 3         ┆ 1106      ┆ Ferdinand de Saussure ┆ Ferdinand de Saussure │
│ 3         ┆ 53        ┆ Arménie               ┆ Arménie               │
│ 3         ┆ 1054   

In [6]:
# Compare similarity vs hyperlink graphs
def analyze_graphs(similarity_path: str, hyperlink_path: str):
    """Compare semantic similarity graph with actual hyperlink graph"""
    
    sim_df = pl.read_csv(similarity_path)
    hyper_df = pl.read_csv(hyperlink_path)
    
    print("📊 Graph Analysis:")
    print(f"Similarity edges: {len(sim_df):,}")
    print(f"Hyperlink edges: {len(hyper_df):,}")
    
    # Find overlap - edges that exist in both graphs
    sim_edges = sim_df.select(["source_id", "target_id"]).with_columns(
        pl.concat_str([pl.col("source_id"), pl.col("target_id")], separator="_").alias("edge")
    )
    hyper_edges = hyper_df.select(["source_id", "target_id"]).with_columns(
        pl.concat_str([pl.col("source_id"), pl.col("target_id")], separator="_").alias("edge")
    )
    
    overlap = sim_edges.join(hyper_edges, on="edge", how="inner")
    print(f"Overlapping edges: {len(overlap):,}")
    print(f"Overlap percentage: {len(overlap) / len(hyper_edges) * 100:.1f}%")
    
    # Find potential missing links (high similarity but no hyperlink)
    missing_links = sim_edges.join(hyper_edges, on="edge", how="anti")
    print(f"Potential missing links: {len(missing_links):,}")
    
    return {
        "similarity_df": sim_df,
        "hyperlink_df": hyper_df,
        "overlap_df": overlap,
        "missing_links_df": missing_links
    }

# Run analysis
results = analyze_graphs("similarity_links_hybrid_test.csv", "hyperlink_graph.csv")

📊 Graph Analysis:
Similarity edges: 5,500
Hyperlink edges: 5,383
Overlapping edges: 402
Overlap percentage: 7.5%
Potential missing links: 5,319


In [ ]:
def print_hyperlink_graph_with_titles(hyperlink_path: str, df: pl.DataFrame, num_rows: int = 20):
    """
    Print hyperlink graph with actual article titles instead of just IDs
    """
    # Load hyperlink graph
    hyper_df = pl.read_csv(hyperlink_path)
    
    # Create ID to title mapping for fast lookup
    id_to_title = dict(zip(df.select("id").to_series(), df.select("title").to_series()))
    
    print("🔗 Hyperlink Graph with Article Titles:")
    print("=" * 80)
    
    for i, row in enumerate(hyper_df.head(num_rows).iter_rows(named=True)):
        source_id = row["source_id"]
        target_id = row["target_id"]
        anchor_text = row["anchor_text"]
        
        source_title = id_to_title.get(source_id, f"Unknown ID {source_id}")
        target_title = id_to_title.get(target_id, f"Unknown ID {target_id}")
        
        print(f"{i+1:2d}. '{source_title}' (ID: {source_id})")
        print(f"    └─→ '{target_title}' (ID: {target_id})")
        print(f"        Anchor: '{anchor_text}'")
        print()
    
    # Summary statistics
    print(f"\n📊 Summary:")
    print(f"Total hyperlinks: {len(hyper_df):,}")
    print(f"Unique source articles: {hyper_df.select('source_id').n_unique():,}")
    print(f"Unique target articles: {hyper_df.select('target_id').n_unique():,}")
    
    # Most linked-to articles
    print(f"\n🎯 Most Referenced Articles:")
    top_targets = (hyper_df
                   .group_by("target_id")
                   .len()
                   .sort("len", descending=True)
                   .head(10))
    
    for row in top_targets.iter_rows(named=True):
        target_id = row["target_id"]
        count = row["len"]
        title = id_to_title.get(target_id, f"Unknown ID {target_id}")
        print(f"  • '{title}' (ID: {target_id}) - {count} incoming links")
    
    # Most linking articles
    print(f"\n📤 Articles with Most Outgoing Links:")
    top_sources = (hyper_df
                   .group_by("source_id")
                   .len()
                   .sort("len", descending=True)
                   .head(10))
    
    for row in top_sources.iter_rows(named=True):
        source_id = row["source_id"]
        count = row["len"]
        title = id_to_title.get(source_id, f"Unknown ID {source_id}")
        print(f"  • '{title}' (ID: {source_id}) - {count} outgoing links")

# Usage
print_hyperlink_graph_with_titles("hyperlink_graph.csv", df, num_rows=15)

🔗 Hyperlink Graph with Article Titles:
 1. 'Antoine Meillet' (ID: 3)
    └─→ 'Allier (département)' (ID: 33)
        Anchor: 'Allier'

 2. 'Antoine Meillet' (ID: 3)
    └─→ 'Cher (département)' (ID: 548)
        Anchor: 'Cher'

 3. 'Antoine Meillet' (ID: 3)
    └─→ 'Allier (département)' (ID: 33)
        Anchor: 'bourbonnaise'

 4. 'Antoine Meillet' (ID: 3)
    └─→ 'Cher (département)' (ID: 548)
        Anchor: 'Cher'

 5. 'Antoine Meillet' (ID: 3)
    └─→ 'Ferdinand de Saussure' (ID: 1106)
        Anchor: 'Ferdinand de Saussure'

 6. 'Antoine Meillet' (ID: 3)
    └─→ 'Arménie' (ID: 53)
        Anchor: 'Arménie'

 7. 'Antoine Meillet' (ID: 3)
    └─→ 'Émile Benveniste' (ID: 1054)
        Anchor: 'Émile Benveniste'

 8. 'Antoine Meillet' (ID: 3)
    └─→ 'Aurélien Sauvageot' (ID: 251)
        Anchor: 'Aurélien Sauvageot'

 9. 'Antoine Meillet' (ID: 3)
    └─→ 'Ferdinand de Saussure' (ID: 1106)
        Anchor: 'Ferdinand de Saussure'

10. 'Antoine Meillet' (ID: 3)
    └─→ 'Bosnie-Herzégov

: 

In [ ]:
def explore_article_links(df: pl.DataFrame, article_id: int, hyperlink_path: str):
    """
    Explore all outgoing and incoming links for a specific article
    """
    hyper_df = pl.read_csv(hyperlink_path)
    id_to_title = dict(zip(df.select("id").to_series(), df.select("title").to_series()))
    
    article_title = id_to_title.get(article_id, f"Unknown ID {article_id}")
    print(f"🔍 Exploring links for: '{article_title}' (ID: {article_id})")
    print("=" * 60)
    
    # Outgoing links (what this article links to)
    outgoing = hyper_df.filter(pl.col("source_id") == article_id)
    print(f"\n📤 Outgoing Links ({len(outgoing)}):")
    for row in outgoing.iter_rows(named=True):
        target_title = id_to_title.get(row["target_id"], f"Unknown ID {row['target_id']}")
        print(f"  → '{target_title}' (anchor: '{row['anchor_text']}')")
    
    # Incoming links (what articles link to this one)
    incoming = hyper_df.filter(pl.col("target_id") == article_id)
    print(f"\n📥 Incoming Links ({len(incoming)}):")
    for row in incoming.iter_rows(named=True):
        source_title = id_to_title.get(row["source_id"], f"Unknown ID {row['source_id']}")
        print(f"  ← '{source_title}' (anchor: '{row['anchor_text']}')")

# Example: Explore links for article ID 3 (Antoine Meillet from your data)
explore_article_links(df, 3, "hyperlink_graph.csv")

# Example: Explore links for article ID 15 (Autriche from your data)
explore_article_links(df, 15, "hyperlink_graph.csv")